In [ ]:
from csc4585 import *

In [ ]:
%%writec pth0.c
#include <stdio.h>
#include <pthread.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
    for(int j=0;j<100;j++) {
        pthread_t p;
        // create and start a thread
        pthread_create(&p,0,task,0);
    }
    return 0;
}

In [ ]:
%%writec pth1.c
#include <stdio.h>
#include <pthread.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
    for(int j=0;j<100;j++) {
        pthread_t p;
        pthread_create(&p,0,task,0);
        // Immediately wait for the thread
        // we just launched, effectively
        // runs sequentially
        pthread_join(p,0);
    }
    return 0;
}

In [ ]:
%%writec pth2.c
#include <stdio.h>
#include <pthread.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
  for(int i=0;i<100;i++) {
    pthread_t p;
    pthread_create(&p,0,task,0);
    // detach tells the runtime we are
    // not going to join the thread
    pthread_detach(p);
  }
  return 0;
}

In [ ]:
%%writec pth3.c
#include <stdio.h>
#include <pthread.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
    
    // Use an pthread_attr_t data structure
    // to tell pthreads we are detaching.
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setdetachstate(&attr,
        PTHREAD_CREATE_DETACHED);
    
    for(int i=0;i<100;i++) {
        pthread_t p;
        pthread_create(&p,&attr,task,0);
    }
    return 0;
}

In [ ]:
%%writec pth4.c
#include <stdio.h>
#include <pthread.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setdetachstate(&attr,PTHREAD_CREATE_DETACHED);
    for(int i=0;i<100;i++) {
        pthread_t p;
        pthread_create(&p,&attr,task,0);
    }
    
    // Free memory in the attr data structure.
    pthread_attr_destroy(&attr);
    return 0;
}

In [ ]:
%%writec pth5.c
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <assert.h>
#include <bits/local_lim.h>

int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++)
        i++;
    printf("i=%d\n",i);
    return 0;
}

int main() {
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    // Use the smallest stack possible.
    pthread_attr_setstacksize(&attr,PTHREAD_STACK_MIN);

    const int N = 100;
    pthread_t p[N];
    for(int i=0;i<N;i++) {
        pthread_create(&p[i],&attr,task,0);
    }
    pthread_attr_destroy(&attr);
    for(int i=0;i<N;i++) {
        // Join each thread from an array
        pthread_join(p[i],0);
    }
    return 0;
}

In [ ]:
%%writec pth6.c
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <assert.h>
#include <bits/local_lim.h>

pthread_mutex_t mut;
int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++) {
        // Use a mutex to lock and unlock
        // exclusive access to the variable i
        pthread_mutex_lock(&mut);
        i++;
        pthread_mutex_unlock(&mut);
    }
    printf("i=%d\n",i);
    return 0;
}

int main() {
    pthread_mutex_init(&mut,0);

    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setstacksize(&attr,PTHREAD_STACK_MIN);

    const int N = 100;
    pthread_t p[N];
    for(int i=0;i<N;i++) {
        pthread_create(&p[i],&attr,task,0);
    }
    pthread_attr_destroy(&attr);
    for(int i=0;i<N;i++) {
        pthread_join(p[i],0);
    }
    return 0;
}

In [ ]:
%%writec pth7.c
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <assert.h>
#include <bits/local_lim.h>

pthread_mutex_t mut;
int i = 0;
void *task(void *v) {
    for(int j=0;j<1000;j++) {
        pthread_mutex_lock(&mut);
        i++;
        pthread_mutex_unlock(&mut);
    }
    printf("i=%d\n",i);
    return 0;
}

int main() {
    pthread_mutex_init(&mut,0);

    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setstacksize(&attr,PTHREAD_STACK_MIN);

    const int N = 100;
    pthread_t p[N];
    for(int i=0;i<N;i++) {
        pthread_create(&p[i],&attr,task,0);
    }
    pthread_attr_destroy(&attr);
    for(int i=0;i<N;i++) {
        pthread_join(p[i],0);
    }
    // This print statement is all that's new...
    printf("AT FINISH: %d\n",i);
    return 0;
}

In [ ]:
%%writec pth8.c
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <assert.h>
#include <bits/local_lim.h>

pthread_mutex_t mut;

// add a condition variable
pthread_cond_t cond;

int i = 0;
void *task(void *v) {
    pthread_mutex_lock(&mut);
    for(int j=0;j<1000;j++)
        i++;
    
    // notify all...
    pthread_cond_broadcast(&cond);
    pthread_mutex_unlock(&mut);
    printf("i=%d\n",i);
    return 0;
}

void *fin_task(void *v) {
    pthread_mutex_lock(&mut);
    while(i < 100000)
        pthread_cond_wait(&cond,&mut);
    pthread_mutex_unlock(&mut);
    printf("AT FINISH: %d\n",i);
}

int main() {
    pthread_mutex_init(&mut,0);
    pthread_cond_init(&cond,0);

    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setstacksize(&attr,PTHREAD_STACK_MIN);

    pthread_t fin;
    pthread_create(&fin,&attr,fin_task,0);

    const int N = 100;
    pthread_t p[N];
    for(int i=0;i<N;i++) {
        pthread_create(&p[i],&attr,task,0);
    }
    pthread_attr_destroy(&attr);
    for(int i=0;i<N;i++) {
        pthread_join(p[i],0);
    }
    pthread_join(fin,0);
    return 0;
}

In [ ]:
%%writec pth9.c
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <assert.h>
#include <bits/local_lim.h>

pthread_mutex_t mut;
pthread_cond_t cond;
int i = 0;

// Use symbolic constants
const int N = 100;
const int N_INC = 1000;

void *task(void *v) {
    pthread_mutex_lock(&mut);
    for(int j=0;j<N_INC;j++)
        i++;
    if(i==N*N_INC)
        pthread_cond_broadcast(&cond);
    pthread_mutex_unlock(&mut);
    printf("i=%d\n",i);
    return 0;
}

void *fin_task(void *v) {
    pthread_mutex_lock(&mut);
    while(i < N*N_INC)
        pthread_cond_wait(&cond,&mut);
    pthread_mutex_unlock(&mut);
    printf("AT FINISH: %d\n",i);
}

int main() {
    pthread_mutex_init(&mut,0);
    pthread_cond_init(&cond,0);

    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setstacksize(&attr,PTHREAD_STACK_MIN);

    pthread_t fin;
    pthread_create(&fin,&attr,fin_task,0);

    pthread_attr_setdetachstate(&attr,PTHREAD_CREATE_DETACHED);
    for(int i=0;i<N;i++) {
        pthread_t p;
        pthread_create(&p,&attr,task,0);
    }
    pthread_attr_destroy(&attr);
    pthread_join(fin,0);
    return 0;
}